In [1]:
import timeit
import numpy as np
import torch

from openmm import *
from openmm.unit import *
from openmm.app import *

import pickle


def investigate(obj):
    return list(filter(lambda x: '__' not in x, dir(obj)))


def size(obj):
    return len(pickle.dumps(obj))


R = 8.314e-3
kBT = R * 300


def state2energy(state):
    energy = state.getPotentialEnergy().value_in_unit(
    kilojoule / mole) / kBT

    # get forces
    force = -state.getForces(asNumpy=True).value_in_unit(
        kilojoule / mole / nanometer) / kBT

    return energy, force

parameter_topology = AmberPrmtopFile('task/energies/proteins/aldp.prmtop')

input_coordinate = AmberInpcrdFile('task/energies/proteins/aldp.crd')

system = parameter_topology.createSystem(implicitSolvent=OBC1, constraints=None, nonbondedCutoff=None, hydrogenMass=None)

In [14]:
system.getNumConstraints()

0

In [ ]:
context = Context(
    system,
    LangevinIntegrator(
        300 * kelvin,
        1. / picosecond,
        1. * femtosecond
    ),
    Platform.getPlatformByName('CUDA'),
)

def energy_evaluation():
    context.setPositions(input_coordinate.positions)
    state = context.getState(getEnergy=True, getForces=True)

    return state2energy(state)

energy_evaluation()

In [4]:
forcefield = ForceField('amber99sb.xml', 'implicit/obc1.xml')
system2 = forcefield.createSystem(parameter_topology.topology)

In [ ]:
context = Context(
    system,
    LangevinIntegrator(
        300 * kelvin,
        1. / picosecond,
        1. * femtosecond
    ),
    Platform.getPlatformByName('CUDA'),
)

def energy_evaluation():
    context.setPositions(input_coordinate.positions)
    state = context.getState(getEnergy=True, getForces=True)

    return state2energy(state)

energy_evaluation()